In [11]:
from __future__ import print_function, division
from bs4 import BeautifulSoup
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import collections
import time

In [9]:
# Create functions to apply to the dataframe
def clean_text(row):                            #Extract Text
    return row.text

def clean_num(row):                             #Extract Number and strip non numerical parts
    return row.text.replace('$','').replace(',','')

In [24]:
counter = 1

for k in range(0,6):
    movie_data = []
    for j in range(0,10):
        response = requests.get('https://www.the-numbers.com/movie/budgets/all/{0}01'.format(k*10+j))
        #print(response.status_code)            #print status code if necessary

        webpage = response.text
        soup = BeautifulSoup(webpage, "lxml")
        rows_soup = soup.find_all('tr')         #this is the soup that contains just the rows I want

        for i in range(1,len(rows_soup),2):     #the rows are formatted such that every other contains movie data
            row_data = []
            data_soup = rows_soup[i].find_all('td')
            for data in data_soup:
                row_data.append(data)
            movie_data.append(row_data)

        time.sleep(5)                            #this is a delay in the event there is throttling of website access

    #turn big movie list into DataFrame
    movie_df = pd.DataFrame(movie_data) 

    #Rename columns in the dataframe to the names in the table/chart
    movie_df = movie_df.rename({0:'rank', 1: "open_date", 2: "title", 3: "prod_budget", 
                                4:'domestic', 5: "worldwide"}, axis='columns')

    movie_df['rank']=movie_df['rank'].apply(clean_text)
    movie_df['open_date']=movie_df['open_date'].apply(clean_text)
    movie_df['title']=movie_df['title'].apply(clean_text)
    movie_df['prod_budget']=movie_df['prod_budget'].apply(clean_num)
    movie_df['domestic']=movie_df['domestic'].apply(clean_num)
    movie_df['worldwide']=movie_df['worldwide'].apply(clean_num)

    #Save CSV with filename 'year_movies.csv'
    movie_df.to_csv('../04_Data/budget_{}.csv'.format(counter)) 
    counter = counter + 1